In [0]:
!pip3 -qq install torch==0.4.1
!pip -qq install torchtext==0.3.1
!pip -qq install spacy==2.0.16
!pip -qq install gensim==3.6.0
!pip -qq install allennlp==0.7.2
!pip -qq install pytorch-pretrained-bert==0.1.2
!python -m spacy download en

!git clone https://github.com/rowanz/swagaf.git
!wget -O conll_2003.zip -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1z7swIvfWs97lKkTpHKIeV7Cgv4NKxGC0"
!unzip conll_2003.zip
!wget -qq https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week08_multitask/conlleval.py
!wget -O fintech.zip -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=110Mi9nF0J_FTv1MHhf1G-FsZIWEErMzK"
!unzip fintech.zip

In [0]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
    DEVICE = torch.device('cuda')
else:
    from torch import FloatTensor, LongTensor
    DEVICE = torch.device('cpu')

np.random.seed(42)

# Pretrained Models

Один из самых заметных трендов в NLP в 2018 году - использование предобученных языковых моделей как составных частей в моделях, тренируемых под конкретные задачи.

Вообще-то мы уже хорошо знакомы с таким подходом: word2vec - это тоже языковая модель, но очень простая. Разница в том, что от эмбеддингов слов без контекста мы переходим к эмбеддингам слов в контексте.

Достаточно пафосное объяснение, почему это важно: [NLP's ImageNet moment has arrived](http://ruder.io/nlp-imagenet/).

При этом возрастает качество - и падает скорость.

## Universal Sentence Encoder

Начнем немного не с языковой модели: [Universal Sentence Encoder](https://arxiv.org/pdf/1803.11175.pdf). Данная модель предобучалась в режиме multi-task learning: энкодер учился выдавать "универсальные" представления предложений, по которым специфичные для каждой задачи декодеры учились  таким вещам как предсказание предыдущего и следующего предложения (Skip-Thoughts like модель) или обычной классификации на размеченных данных.

Основной понт в том, что представления получаются вполне себе интепретируемые даже без какого-то специального дообучения под нужную задачу.

*К сожалению, я не умею в эту модель на pytorch, поэтому придется с tensorflow жить...*

In [0]:
import tensorflow as tf
import tensorflow_hub as hub


tf.reset_default_graph()
sess = tf.InteractiveSession()

universal_sentence_encoder = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2", trainable=False)
sess.run([tf.global_variables_initializer(), tf.tables_initializer()])

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

inputs = tf.placeholder(tf.string, shape=[None])
outputs = universal_sentence_encoder(inputs)

lines = [
    "How old are you?",                                                                 # 0
    "Attempting to light a cigarette, someone fumbles with the lighter and drops it.",  # 1
    "This is the story of a man named Neil Fisk, and how he came to love God.",         # 2
    "What is your age?",                                                                # 3
    "Do you have a moment to talk about our Lord?",                                     # 4
]

result = sess.run(outputs, {
    inputs: lines
})

plt.title('phrase similarity')
plt.imshow(result.dot(result.T), interpolation='none', cmap='gray')

Например, фразы "How old are you?" и "What is your age?" не имеют общих слов, но их косинусная близость достаточно высока. Аналогично со второй и четвертой фразами.

Обратите внимание, что векторы на выходе из модели уже нормированы - поэтому косинусная близость считается как скалярное произведение.

Как пример использования этих представлений почти бесплатно, порешаем такую (упоротую) задачу, утащенную у другого курса: https://www.kaggle.com/c/fintech-tinkoff

In [0]:
import pandas as pd

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

Дано 60 тысяч пар похожих вопросов + куча вопросов из категории "другое". Нужно определить, к какой паре ближе данный вопрос или сказать, что он из другого.

In [0]:
train_data.iloc[:10]

Модель вполне себе умеет говорить, что вопросы похожи!

In [0]:
result = sess.run(outputs, {
    inputs: train_data.iloc[:10].text
})

plt.title('phrase similarity')
plt.imshow(result.dot(result.T), interpolation='none', cmap='gray')

Напишем функцию для подсчета эмбеддингов всех предложений в датасете:

In [0]:
def calc_vectors(data):
    BATCH_SIZE = 1024

    vectors = []
    for batch_begin in range(0, len(data), BATCH_SIZE):
        batch_end = min(len(data), batch_begin + BATCH_SIZE)

        vectors.append(
            sess.run(outputs, {
                inputs: data.iloc[batch_begin: batch_end].text
            })
        )

    return np.concatenate(vectors, 0)

In [0]:
train_vectors = calc_vectors(train_data)
train_labels = train_data['labels'].values

test_vectors = calc_vectors(test_data)

In [0]:
for test_ind in range(10):
    print(test_data.iloc[test_ind].text, train_data.iloc[(train_vectors * test_vectors[test_ind]).sum(-1).argmax()].text, sep='\t')

**Задание** У нас есть база из векторов и соответствующих им меток. Реализуйте 1NN - поиск ближайшего соседа для запросов из тестовой выборки.

В качестве метки тогда можно взять метку этого ближайшего соседа.

Вообще, чтобы делать поиск быстро по большой базе применяют приближенные алгоритмы типа: [HNSW](https://github.com/nmslib/hnswlib). В данном случае это не актуально, но всё равно можно попробовать заменить свой 1NN на 2NN из той библиотеки.

## ELMo

Другая история в случае с [ELMo](https://arxiv.org/pdf/1802.05365.pdf). Это обычная языковая модель:

![](https://i.ibb.co/dpp00wG/elmo.png)  
*From [Improving a Sentiment Analyzer using ELMo — Word Embeddings on Steroids](http://www.realworldnlpbook.com/blog/improving-sentiment-analyzer-using-elmo.html)*

Ну, почти обычная. Во-первых, эмбеддинги слов строятся с помощью сверточной сети над символами.

Во-вторых, учится сразу две языковых модели, forward и backward, которые потом конкатенируются.

Наконец, в-третьих, смысл модели, что она выдает эмбеддинг слова с учетом его контекста. Это мог бы быть выход последнего слоя LSTM, но ребята поступили хитрее: для каждого слова у нас есть сразу несколько эмбеддингов: выходы каждого из слоев LSTM + выход сверточной сети над символами. При обучении итоговой модели под нужную задачу эмбеддинг слова считается как взвешенная сумма данных эмбеддингов. Веса учатся под задачу.

В итоге, когда мы хотим использовать ELMo в своей модели, мы берем эту вот языковую модель и подставляем вместо наших обычных эмбеддингов. Всё, пара строчек изменений - но более хорошие и сильно более медленные эмбеддинги.

Языковую модель можно доучивать под задачу - тогда придется дообучать все эти миллионы параметров, что медленно. А можно не доучивать - тогда внутри ELMo будет учится только `(num_layers + 1)` параметр - веса смеси эмбеддингов.

### NER

Такие эмбеддинги можно использовать где угодно, но лучше всего они смотрятся в задачах, связанных с разметкой последовательностей (там критичнее всего получать эмбеддинги с учетом контекста; тогда как Universal Sentence Encoder в задачах классификации смотрится логичнее ELMo).

In [0]:
def read_dataset(path):
    data = []
    with open(path) as f:
        words, tags = [], []
        for line in f:
            line = line.strip()
            if not line and words:
                data.append((words, tags))
                words, tags = [], []
                continue
            word, pos_tag, synt_tag, ner_tag = line.split()
            words.append(word)
            tags.append(ner_tag)
        if words:
            data.append((words, tags))
    return data

In [0]:
train_data = read_dataset('train.txt')
val_data = read_dataset('valid.txt')
test_data = read_dataset('test.txt')

Мы уже смотрели на NER, но вообще он такой:

In [0]:
train_data[:3]

Соберем датасет:

In [0]:
from torchtext.data import Field, Example, Dataset, BucketIterator

tokens_field = Field(unk_token=None, batch_first=True)
tags_field = Field(unk_token=None, batch_first=True)

fields = [('tokens', tokens_field), ('tags', tags_field)]

train_dataset = Dataset([Example.fromlist(example, fields) for example in train_data], fields)
val_dataset = Dataset([Example.fromlist(example, fields) for example in val_data], fields)
test_dataset = Dataset([Example.fromlist(example, fields) for example in test_data], fields)

tokens_field.build_vocab(train_dataset, val_dataset, test_dataset)
tags_field.build_vocab(train_dataset)

print('Vocab size =', len(tokens_field.vocab))
print('Tags count =', len(tags_field.vocab))

train_iter, val_iter, test_iter = BucketIterator.splits(
    datasets=(train_dataset, val_dataset, test_dataset), batch_sizes=(32, 128, 128), 
    shuffle=True, device=DEVICE, sort=False
)

*Обратите внимание: словарь строится по всем датасетам. Не то, чтобы это очень этично, но мы не будем доучивать никакие эмбеддинги - поэтому ничего нечестного.*

### Baseline

Для начала обучим модель с предобученными словными эмбеддингами:

In [0]:
import gensim.downloader as api

w2v_model = api.load('glove-wiki-gigaword-100')

In [0]:
embeddings = np.zeros((len(tokens_field.vocab), w2v_model.vectors.shape[1]))

for i, token in enumerate(tokens_field.vocab.itos):
    if token.lower() in w2v_model.vocab:
        embeddings[i] = w2v_model.get_vector(token.lower())

**Задание** Допишите модель простого теггера.  
Обратите внимание `batch_first=True` в `fields` (это для ELMo понадобится).

In [0]:
class BaselineTagger(nn.Module):
    def __init__(self, embeddings, tags_count, emb_dim=100, rnn_dim=256, num_layers=1):
        super().__init__()
        
        <init layers>
        
    def forward(self, inputs):
        <apply layers>

**Задание** Допишите тренировщик модели.

In [0]:
class ModelTrainer():
    def __init__(self, model, criterion, optimizer):
        self._model = model
        self._criterion = criterion
        self._optimizer = optimizer
        
    def on_epoch_begin(self, is_train, name, batches_count):
        """
        Initializes metrics
        """
        self._epoch_loss = 0
        self._correct_count, self._total_count = 0, 0
        self._is_train = is_train
        self._name = name
        self._batches_count = batches_count
        
        self._model.train(is_train)
        
    def on_epoch_end(self):
        """
        Outputs final metrics
        """
        return '{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
            self._name, self._epoch_loss / self._batches_count, self._correct_count / self._total_count
        )
        
    def on_batch(self, batch):
        """
        Performs forward and (if is_train) backward pass with optimization, updates metrics
        """        
        loss = <calc loss>
        correct_count, total_count = <and this stuff>
        
        self._correct_count += correct_count
        self._total_count += total_count
        self._epoch_loss += loss.item()
        
        if self._is_train:
            self._optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(self._model.parameters(), 1.)
            self._optimizer.step()

        return '{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
            self._name, loss.item(), correct_count / total_count
        )

Воспользуемся уже знакомой функцией для оценки теггера:

In [0]:
from conlleval import evaluate

def eval_tagger(model, test_iter):
    true_seqs, pred_seqs = [], []

    model.eval()
    with torch.no_grad():
        for batch in test_iter:
            logits = model(batch.tokens)
            preds = logits.argmax(-1)

            seq_lengths = (batch.tags != 0).sum(-1)

            for i, seq_len in enumerate(seq_lengths):
                true_seqs.append(' '.join(tags_field.vocab.itos[ind] for ind in batch.tags[i, :seq_len]))
                pred_seqs.append(' '.join(tags_field.vocab.itos[ind] for ind in preds[i, :seq_len]))

    print('Precision = {:.2f}%, Recall = {:.2f}%, F1 = {:.2f}%'.format(*evaluate(true_seqs, pred_seqs, verbose=False)))

In [0]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(trainer, data_iter, is_train, name=None):
    trainer.on_epoch_begin(is_train, name, batches_count=len(data_iter))
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=len(data_iter)) as progress_bar:
            for i, batch in enumerate(data_iter):
                batch_progress = trainer.on_batch(batch)

                progress_bar.update()
                progress_bar.set_description(batch_progress)
                
            epoch_progress = trainer.on_epoch_end()
            progress_bar.set_description(epoch_progress)
            progress_bar.refresh()

            
def fit(trainer, train_iter, epochs_count=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        do_epoch(trainer, train_iter, is_train=True, name=name_prefix + 'Train:')
        
        if not val_iter is None:
            do_epoch(trainer, val_iter, is_train=False, name=name_prefix + '  Val:')
            eval_tagger(trainer._model, val_iter)
            print(flush=True)

Запустим обучение модели:

In [0]:
model = BasicTagger(len(tokens_field.vocab), tags_count=len(tags_field.vocab)).to(DEVICE)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters())

trainer = ModelTrainer(model, criterion, optimizer)

fit(trainer, train_iter, epochs_count=32, val_iter=val_iter)

### ELMo Model

Возьмем предобученную модель от авторов (описание работы с ней есть в [ELMo how to](https://github.com/allenai/allennlp/blob/master/tutorials/how_to/elmo.md)).

In [0]:
from allennlp.modules.elmo import Elmo, batch_to_ids

options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

elmo = Elmo(options_file, weight_file, num_output_representations=1,
            dropout=0, vocab_to_cache=tokens_field.vocab.itos).to(DEVICE)

Вообще надо сначала преобразовать предложение в символьное представление:

In [0]:
sentences = [['First', 'sentence', '.'], ['Another', '.']]
character_ids = batch_to_ids(sentences).to(DEVICE)

character_ids

А потом засунуть это в модель elmo:

In [0]:
elmo(character_ids)

Нас интересует `elmo_representations`.

Такой интерфейс не слишком удобный - нужно передавать в `batch_to_ids` строки, а это значит - писать с нуля генератор батчей. Кроме этого, плохо передавать большие батчи на gpu - а с символьным представлением из батча `(batch_size, seq_len)` мы получаем батча в `max_word_len` раз больший. Наконец, рассчет словных эмбеддингов по символам не бесплатный (относительно запроса к таблице эмбеддингов).

Поэтому при создании модели мы закэшировали все эмбеддинги: параметр `vocab_to_cache=tokens_field.vocab.itos`.

Чтобы пользоваться закэшированными значениями в `inputs` будем передавать какой-то мусор, а в `word_inputs` - индексы эмбеддингов.

In [0]:
batch = next(iter(train_iter))

elmo(inputs=batch.tokens.new_empty((batch.tokens.shape[0], batch.tokens.shape[1], 50)), 
     word_inputs=batch.tokens)

**Задание** Обновите модель, заменив эмбеддинги на ELMo.

In [0]:
class ELMoTagger(nn.Module):
    def __init__(self, elmo, tags_count, emb_dim=1024, rnn_dim=256, num_layers=1):
        super().__init__()
        
        <init layers>
        
    def forward(self, inputs):
        <apply layers>

In [0]:
model = ELMoTagger(elmo, tags_count=len(tags_field.vocab)).to(DEVICE)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters())

trainer = ModelTrainer(model, criterion, optimizer)

fit(trainer, train_iter, epochs_count=32, val_iter=val_iter)

### CRF

Это не имеет отношение к происходящему, вообще говоря, но тоже полезная информация: чтобы лучше работать с задачей предсказания тегов последовательности можно вместо независимого предсказания каждого тега в отдельности предсказывать тег при условии предыдущего тега.

Это выглядит как декодер в Seq2Seq моделях - только вместо предыдущего слова на вход подается предыдущий тег.

Такое предсказание можно реализовать разными способами. Один из - использование CRF (Conditional Random Field) (Хорошее описание есть [здесь](http://www.davidsbatista.net/blog/2017/11/13/Conditional_Random_Fields/).

У нас сейчас есть модель с полносвязным выходным слоем. На каждом шаге по его выходу оценивается вероятность того, какой тег имеет данное слово - просто нормализуются скоры с помощью softmax: $p[i]= \frac{e^{s[i]}}{\sum_{j=1}^9 e^{s[j]}}$.

А давайте вместо локальной нормализации, делать глобальную, на всю последовательность. Кроме этого, давайте учить вероятности перехода из одного тега на предыдущем шаге в другой тег на следующем $T[y_t, y_{t+1}]$. Например, должно выучиться, что вероятность после `O` встретиться `I-LOC` нулевая (`I-LOC` может быть только после `B-LOC` или другого `I-LOC`).

Тогда каждая последовательность будет оцениваться по такой формуле:
$$\begin{align*}
C(y_1, \ldots, y_m) &= b[y_1] &+ \sum_{t=1}^{m} s_t [y_t] &+ \sum_{t=1}^{m-1} T[y_{t}, y_{t+1}] &+ e[y_m]\\
                    &= \text{begin} &+ \text{scores} &+ \text{transitions} &+ \text{end}
\end{align*}$$

Например, могут быть два варианта последовательности:

![](https://guillaumegenthial.github.io/assets/crf1.png =x200) ![](https://guillaumegenthial.github.io/assets/crf2.png =x200)   
*From [Sequence Tagging with Tensorflow](https://guillaumegenthial.github.io/sequence-tagging-with-tensorflow.html)*

Лучше из них та, у которой ниже сумма скоров тегов + сумма переходов между тегами.

В данном случае нужно просто добавить модуль ConditionalRandomField - он будет выучивать переходы $T$, считать лосс глобальный на всю последовательность, а также делать декодинг.

Подсчет лосса осуществляется таким образом:
```python
crf = ConditionalRandomField(tags_count)
loss = -crf(output, tags, mask)
```
где `output` - выход полносвязного слоя, который раньше был последним.

Декодинг делается так:
``` python
decoded_sequences = crf.viterbi_tags(output, mask)
```

**Задание** Обновите теггер и функции обучения с оценкой качества теггера.

In [0]:
from allennlp.modules import ConditionalRandomField

class CRFTagger(nn.Module):
    def __init__(self, embeddings, tags_count, emb_dim=100, rnn_dim=256, num_layers=1):
        super().__init__()
        <init layers (embeddings can be from either glove or elmo)>
        
    def forward(self, inputs, mask, tags=None):
        <apply layers like in previos models>
        
        if tags is not None:
            return <crf loss>
        return <viterbi decoding>

## BERT

Наконец, третий вариант модели (что-то среднее с кучей своих плюшек) - это BERT (да, чуваки умеют называть свои модели).

Сходство с ELMo - это тоже языковая модель. Отличия:
1. Это Transformer, а не BiLSTM.
![](https://i.ibb.co/PQ4qXtr/2018-12-22-21-13-14.png)
Чтобы учить языковую модель в рамках трансформера они случайно выкидывали некоторые слова из предложения и пытались предсказывать их с помощью сети. Таким образом, при предсказании токена был доступен весь контекст, а не только левый или только правый, как в ELMo.

2. Использовалась дополнительная задача в стиле Skip-Thoughts - предсказание следующего предложения:
![](https://i.ibb.co/WWdwmPD/2018-12-22-21-12-59.png =x250)

Пара предложений записывалась подряд (внимание на SEP) и модель училась предсказывать, идут ли они подряд в реальности (внимание на CLS).

В итоге эмбеддинг CLS выучивался таким, что в нем содержалась информация обо всем контексте - а дальше его легко можно использовать для классификации, как в первой модели. Но при этом выучиваются еще и контекстные эмбеддинги всех слов в предложении, поэтому модель можно использовать аналогично ELMo в задаче теггирования.

BERT замечателен тем, что побил результаты всех существующих на данный момент моделей, а также human performance на некоторых задачах ([например, SQuAD 1](https://rajpurkar.github.io/SQuAD-explorer/)).

Воспользуемся им для задачи [Swag](https://rowanzellers.com/swag/) - выбор правильного продолжения для текста. Датасет, как следует из названия, формировали таким образом, чтобы это было сложно сделать для бездушной машины (но через несколько месяцев вышел BERT и всё насмарку :( ).

In [0]:
import pandas as pd

train_data = pd.read_csv('swagaf/data/train.csv')
val_data = pd.read_csv('swagaf/data/val.csv')

In [0]:
train_data.sample(10)[['startphrase', 'sent1', 'sent2', 'ending0', 'ending1', 'ending2', 'ending3', 'label']]

Обратите внимание на токенизатор - модель работает с подсловами как в bpe.

In [0]:
from pytorch_pretrained_bert import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
print(tokenizer.vocab)

tokens = tokenizer.tokenize('Why do I do this stuff...')
print(tokens)
print(tokenizer.convert_tokens_to_ids(tokens))

Соберем датасет.

In [0]:
def collect_samples(data, tokenizer):
    choices_list, labels_list, segment_ids_list = [], [], []
    
    for _, row in tqdm(data.iterrows(), total=len(data)):
        first_phrase = tokenizer.tokenize(row.sent1)
        second_phrase = tokenizer.tokenize(row.sent2)
        choices, segment_ids = [], []
        for i, ending in enumerate(row[['ending0', 'ending1', 'ending2', 'ending3']]):
            ending = tokenizer.tokenize(ending)
            tokens = ["[CLS]"] + first_phrase + ["[SEP]"] + second_phrase + ending + ["[SEP]"]
            tokens = tokenizer.convert_tokens_to_ids(tokens)
            choices.append(tokens)
            
            segment_ids.append([0] * (len(first_phrase) + 2) + [1] * (len(tokens) - len(first_phrase) - 2))
            
        choices_list.append(choices)
        segment_ids_list.append(segment_ids)
        labels_list.append(row.label)

    return np.array(choices_list), np.array(labels_list), np.array(segment_ids_list)


train_data, train_labels, train_segment_ids = collect_samples(train_data, tokenizer)
val_data, val_labels, val_segment_ids = collect_samples(val_data, tokenizer)

Будем сэмплы формировать также, как было при обучении модели - `[CLS]<first text>[SEP]<next possible text>[SEP]`.

Чтобы модель знала, где заканчивается первый сегмент и начинается второй передается заодно еще `train_segment_ids` - 0 соответствует токену из первого сегмента, а 1 - из второго.

In [0]:
train_data[:2], train_labels[:2], train_segment_ids[:2]

Итератор батчей таки придется написать свой...

In [0]:
import math


def to_matrix(choices_list):
    batch_size = len(choices_list)
    num_options = len(choices_list[0])
    seq_len = max(len(choice) for choices in choices_list for choice in choices)
    
    matrix = np.zeros((batch_size, num_options, seq_len))
    for i, choices in enumerate(choices_list):
        for j, choice in enumerate(choices):
            matrix[i, j, :len(choice)] = choice

    return matrix
    

class BatchIterator():
    def __init__(self, data, labels, segment_ids, batch_size, shuffle=True):
        self._data = data
        self._labels = labels
        self._segment_ids = segment_ids
        self._num_samples = len(data)
        self._batch_size = batch_size
        self._shuffle = shuffle
        self._batches_count = int(math.ceil(len(data) / batch_size))
        
    def __len__(self):
        return self._batches_count
    
    def __iter__(self):
        return self._iterate_batches()

    def _iterate_batches(self):
        indices = np.arange(self._num_samples)
        if self._shuffle:
            np.random.shuffle(indices)

        for start in range(0, self._num_samples, self._batch_size):
            end = min(start + self._batch_size, self._num_samples)

            batch_indices = indices[start: end]
            
            choices = to_matrix(self._data[batch_indices])
            mask = (choices != 0).astype(np.int)
            yield {
                'choices': choices,
                'segment_ids': to_matrix(self._segment_ids[batch_indices]),
                'mask': mask,
                'label': self._labels[batch_indices]
            }

Тренироваться на colab можно только с очень маленьким батчем:

In [0]:
train_iter = BatchIterator(train_data, train_labels, train_segment_ids, 8)
val_iter = BatchIterator(val_data, val_labels, val_segment_ids, 8)

Загружаем предобученный BERT:

In [0]:
from pytorch_pretrained_bert import BertModel

bert = BertModel.from_pretrained('bert-base-uncased').to(DEVICE)

Строим такую модель:  
![](https://i.ibb.co/JFcwW6D/2018-12-22-21-23-20.png =x400)

BERT строит четыре представления $C_0, \ldots, C_3$ для четырех вариантов продолжения предложения. Будем учить параметр $V$, чьё скалярное произведение с $C_i$ должно быть максимальным для релевантного продолжения.

Для этого просто будем использовать кросс-энтропийные потери и считать softmax:
$P_i = \frac{e^{V\cdot C_i}}{\sum_j e^{V\cdot C_j}}$.

То есть прикол в том, что единственное специфичное для задачи, что мы учим - это вектор $V$!

In [0]:
class MultipleChoiceModel(nn.Module):
    def __init__(self, bert, num_choices):
        super().__init__()
        
        self._bert = bert
        self._num_choices = num_choices
        self._dropout = nn.Dropout(0.1)
        self._classifier = nn.Linear(768, 1)
        
    def forward(self, choices, segment_ids, mask):
        """
        choices: LongTensor of shape [batch_size, num_choices, seq_len] with token ids
        segment_ids: LongTensor of shape [batch_size, num_choices, seq_len] with token types (0 for first segment, 1 for second)
        mask: LongTensor of shape [batch_size, num_choices, seq_len] with mask for padding tokens
        returns logits - FloatTensor of shape [batch_size, num_choices]
        """
        choices = choices.view(-1, choices.size(-1))
        segment_ids = segment_ids.view(-1, segment_ids.size(-1))
        mask = mask.view(-1, mask.size(-1))
        
        _, pooled_output = self._bert(choices, segment_ids, mask, output_all_encoded_layers=False)
        
        pooled_output = self._dropout(pooled_output)
        logits = self._classifier(pooled_output)
        return logits.view(-1, self._num_choices)

**Задание** Доделайте обучалку для модели.

In [0]:
class ModelTrainer():
    def __init__(self, model, criterion, optimizer):
        self._model = model
        self._criterion = criterion
        self._optimizer = optimizer
        
    def on_epoch_begin(self, is_train, name, batches_count):
        """
        Initializes metrics
        """
        self._epoch_loss = 0
        self._correct_count, self._total_count = 0, 0
        self._is_train = is_train
        self._name = name
        self._batches_count = batches_count
        
        self._model.train(is_train)
        
    def on_epoch_end(self):
        """
        Outputs final metrics
        """
        return '{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
            self._name, self._epoch_loss / self._batches_count, self._correct_count / self._total_count
        )
        
    def on_batch(self, batch):
        """
        Performs forward and (if is_train) backward pass with optimization, updates metrics
        """
        
        loss = <calc loss>
        correct_count, total_count = <and this stuff>
        
        self._correct_count += correct_count
        self._total_count += total_count
        self._epoch_loss += loss.item()
        
        if self._is_train:
            self._optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(self._model.parameters(), 1.)
            self._optimizer.step()

        return '{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
            self._name, loss.item(), correct_count / total_count
        )

Нужно также немного магии для инициализации оптимизатора:

In [0]:
from pytorch_pretrained_bert.optimization import BertAdam

model = MultipleChoiceModel(bert, 4).to(DEVICE)
criterion = nn.CrossEntropyLoss()

params = [(name, param) for name, param in model.named_parameters() if 'pooler' not in name]

no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [param for name, param in params if not any(nd in name for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [param for name, param in params if any(nd in name for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = BertAdam(optimizer_grouped_parameters, lr=5e-5, warmup=0.1, t_total=len(train_iter) * 2)

trainer = ModelTrainer(model, criterion, optimizer)

fit(trainer, train_iter, 2, val_iter)

**Задание** В память не влезает больший батч. А хочется тренировать с большим батчем (у гугла был 16).

Чтобы решить это, есть такой простой (если на pytorch, а не на tensorflow) прием - накапливание градиентов. Можно просто оптимизировать модель не на каждом шаге, а раз в несколько шагов обучения:
[Training Neural Nets on Larger Batches: Practical Tips for 1-GPU, Multi-GPU & Distributed setups](https://medium.com/huggingface/training-larger-batches-practical-tips-on-1-gpu-multi-gpu-distributed-setups-ec88c3e51255)

Реализуйте такой подход.

# Дополнительные материалы

## Статьи
Universal Language Model Fine-tuning for Text Classification [[pdf]](https://arxiv.org/pdf/1801.06146)  
Deep contextualized word representations [[pdf]](https://arxiv.org/pdf/1802.05365)  
Improving Language Understanding by Generative Pre-Training [[pdf]](https://s3-us-west-2.amazonaws.com/openai-assets/research-covers/language-unsupervised/language_understanding_paper.pdf)  
BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding [[pdf]](https://arxiv.org/pdf/1810.04805.pdf)

Dissecting Contextual Word Embeddings: Architecture and Representation [[pdf]](http://aclweb.org/anthology/D18-1179)

## Блоги
[NLP's ImageNet moment has arrived](http://ruder.io/nlp-imagenet/)  
[The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)](https://jalammar.github.io/illustrated-bert/)  
[Improving a Sentiment Analyzer using ELMo — Word Embeddings on Steroids](http://www.realworldnlpbook.com/blog/improving-sentiment-analyzer-using-elmo.html)

[Sequence Tagging with Tensorflow](https://guillaumegenthial.github.io/sequence-tagging-with-tensorflow.html)  
[Conditional Random Field Tutorial in PyTorch](https://towardsdatascience.com/conditional-random-field-tutorial-in-pytorch-ca0d04499463)  
[Conditional Random Fields for Sequence Prediction](http://www.davidsbatista.net/blog/2017/11/13/Conditional_Random_Fields/)

# Сдача

[Форма для сдачи](https://goo.gl/forms/UvDRQh1CAR3R90No2)  
[Feedback](https://goo.gl/forms/9aizSzOUrx7EvGlG3)